# ONLINE SHOPPERS INTENTION

### Predicting conversion of revenue based on customer’s activity

## DATA SOURCE
 The dataset is obtained from Google Analytics which gathers information about the performance of websites and also all the activities carried out on the website by the customers. The dataset has 12330 unique sessions over a period of 12 months that can be used to predict the behaviour of customers on the website. The target variable is imbalanced with a proportion of majority class (True) being 84.5% and False being 15.5%. The dataset has parameters of google analytics that can be used to predict the revenue generated by customers.


### DATA DESCRIPTION
### CONTINUOS FEATURES


Administrative	     -Integer	-Number of different pages visited related 
                                to the administrative concerns of the website	                    

Informational	     -Integer	-Number of different pages visited related to the information                                                                   of the website and other useful contents of the website          

ProductRelated	    - Integer	-Number of different pages visited related to different 
                                products of the website.	                                        

BounceRate         	- Float	 -   Percentage of users who left the website from the landing page	     

ExitRate	        - Float	  -  Percentage of users who left from the page the visit	             

Page Values	        - Float	   - Page Value is the average value for a page that a user 
                                visited before making a transaction.	                             

SpecialDay	        - Float	    
                               
                               0 – day of the session is not within 10 days of a special day.
                               Between 0.1 and 0.5 – day of the session is between 10 days and 5 days away from a special day.
                               Between 0.6 and 0.9 – day of the session is between 4 days and 1 day away from a special day.
                               1 – day of the session is a special day.	

The features Bounce Rates, Exit Rates and Page Values are terms of Google analytics that are used to determine the performance of a website. The bounce rate is used to show the percentage of single page sessions out of all the sessions. The exit rate shows us the percentage of people who left from a page. Page value gives us the average value of the transaction to all the pages visited which is equally distributed to all pages.

### CATEGORICAL FEATURES


Administrative_Duration	   -Integer	    
                                        
    Time spent on Administrative pages in seconds
                                        0 – No time was spent on the page                                 
                                        1 - Less time of under 233 was spent on the page
                                        2 – More time of  over 233 was spent on the page	

Informational_duration	   Integer	    

    Time spent on Informational pages in seconds                       
                                          0 – No time was spent on the page
                                          1- More time was spent on the page	

ProductRelated_Duration    Integer	    

    Time spent on pages related to products in seconds                 
                                          0 – No time was spent on the page
                                          1 - Less time of under 3384 was spent on the page
                                          2 – More time of  over 3384 was spent on the page	

Browse                    -  Integer    	-ID of bowsers from which the session took place	                   

Region	                   - Integer	 -   ID of Regions from which the session took place	

Traffic Type	            -Integer	-    ID of different types of sources from which the users landed on the website	

User Type	                -String	   - Whether the user is a returning user or a new user or of any other type	

Revenue	                   - Boolean	-    Whether the user contributed to the revenue by purchasing or not	

Weekend	                  -  Boolean  -	Whether the session was on a weekend or not	

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
# dataframe display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# filtering warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#Carregar dados
df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv")
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df_new=df.copy(deep=True)

In [ ]:
#administration duration
q1=df['Administrative_Duration'].quantile(0.25)
q3=df['Administrative_Duration'].quantile(0.75)
iqr=q3-q1
u_l1=q3+1.5*iqr

In [ ]:
df_new['Administrative_Duration']=df_new['Administrative_Duration'].map(lambda x:0 if x==0 else (1 if x>0 and x<u_l1 else 2))
df_new['Administrative_Duration'].value_counts()

In [ ]:
df_new['Informational_Duration']=df_new['Informational_Duration'].map(lambda x:0 if x==0 else 1)
df_new['Informational_Duration'].value_counts()

In [ ]:
#ProductRelated_Duration
q1=df['ProductRelated_Duration'].quantile(0.25)
q3=df['ProductRelated_Duration'].quantile(0.75)
iqr=q3-q1
u_l1=q3+1.5*iqr

In [ ]:
df_new['ProductRelated_Duration']=df_new['ProductRelated_Duration'].map(lambda x:0 if x==0 else (1 if x>0 and x<u_l1 else 2))
df_new['ProductRelated_Duration'].value_counts()

In [ ]:
df_new.shape

In [ ]:
df_new['Revenue'].value_counts()

# Administrative_Duration

In [ ]:
ct=pd.crosstab(df_new['Administrative_Duration'],df_new['Revenue'],values=df_new['BounceRates'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

we can see avg.bounce rate when revenue true is very less then when it is false

even avg.bounce rate is decreasing as time spent increases.

In [ ]:
ct=pd.crosstab(df_new['Administrative_Duration'],df_new['Revenue'],values=df_new['ExitRates'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

we can see avg.Exit rate when revenue true is very less then when it is false

even avg.Exit rate is decreasing as time spent increases.

In [ ]:
ct=pd.crosstab(df_new['Administrative_Duration'],df_new['Revenue'],values=df_new['PageValues'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

we can see pagevalue is high at duration 0 it is because of other pages

# Informational_Duration

In [ ]:
ct=pd.crosstab(df_new['Informational_Duration'],df_new['Revenue'],values=df_new['BounceRates'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

avg.bounce rate when revenue is true is very less than when revenue is true

In [ ]:
ct=pd.crosstab(df_new['Informational_Duration'],df_new['Revenue'],values=df_new['ExitRates'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

avg.exit rate when revenue is true is very less than when revenue is true

even avg.Exit rate is decreasing as time spent increases.

In [ ]:
ct=pd.crosstab(df_new['Informational_Duration'],df_new['Revenue'],values=df_new['PageValues'],aggfunc='mean')
ct

In [ ]:
ct.plot.bar()

we can see high pagevalue for duration 0 it is because of other pages when this is 0

# ProductRelated_Duration

In [ ]:
ct=pd.crosstab(df_new['ProductRelated_Duration'],df_new['Revenue'],values=df_new['BounceRates'],aggfunc='mean')

In [ ]:
ct.plot.bar()

we can see avg.bounce rate when revenue true is very less then when it is false

even avg.bounce rate is decreasing as time spent increases.

In [ ]:
ct=pd.crosstab(df_new['ProductRelated_Duration'],df_new['Revenue'],values=df_new['ExitRates'],aggfunc='mean')

In [ ]:
ct.plot.bar()

we can see avg.exit rate when revenue true is very less then when it is false

even avg.exit rate is decreasing as time spent increases.

In [ ]:
ct=pd.crosstab(df_new['ProductRelated_Duration'],df_new['Revenue'],values=df_new['PageValues'],aggfunc='mean')

In [ ]:
ct.plot.bar()

page values are very high for product related than other  

we can say product related is mostly preffered over other 

In [ ]:
# count of sessions for each duration

In [ ]:
sns.countplot(x=df_new['Administrative_Duration'], hue=df['Revenue'])

In [ ]:
sns.countplot(x=df_new['Informational_Duration'], hue=df['Revenue'])


In [ ]:
sns.countplot(x=df_new['ProductRelated_Duration'], hue=df['Revenue'])


we can see when sessions are product related then most of the revenue is generated 

most the time is spent on product related.

informational pages are mostly not interested 

# Region 

### region n duration vs page value

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['Administrative_Duration'],values=df_new['PageValues'],aggfunc='mean')
ct.plot.bar()

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['Informational_Duration'],values=df_new['PageValues'],aggfunc='mean')
ct.plot.bar()

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['ProductRelated_Duration'],values=df_new['PageValues'],aggfunc='mean')
ct.plot.bar()

### region n duration vs bounce rate

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['Administrative_Duration'],values=df_new['BounceRates'],aggfunc='mean')
ct.plot.bar()

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['Informational_Duration'],values=df_new['BounceRates'],aggfunc='mean')
ct.plot.bar()

In [ ]:
ct=pd.crosstab(df_new['Region'],df_new['ProductRelated_Duration'],values=df_new['BounceRates'],aggfunc='mean')
ct.plot.bar()

In [ ]:
df_new.head()

In [ ]:
df1=pd.get_dummies(data=df_new,columns=['Month','OperatingSystems','Browser','Region','TrafficType','VisitorType','Weekend','Revenue'],drop_first=True)
df1.head()

In [ ]:
df1.rename(columns={'Revenue_True':'Revenue'},inplace=True)
df1.shape

In [ ]:
df_p=df1.copy(deep=True)

# modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report,confusion_matrix


In [ ]:
X=df_p.drop('Revenue',axis=1)
y=df_p['Revenue']

# splitting the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=5)

In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

# logistic regression

Logistic regression is a type of regression analysis in statistics used for prediction of outcome of a categorical dependent variable from a set of predictor or independent variables. In logistic regression the dependent variable is always binary. Logistic regression is mainly used to for prediction and also calculating the probability of success. 

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

# confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

In [ ]:
cm_reference = pd.DataFrame(np.array(["TP","FP","FN","TN"]).reshape(2,2), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
print(cm_reference)

In [ ]:
TP=cm[0,0]
TN=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

In [ ]:
print("True Negatives :",TN)
print("True Positives :",TP)
print("False Negative :",FN," (Type II error)")
print("False Positives :",FP," (Type I error)")
print("correctly predicted :",TP+TN)
print("miss-classified :",FN+FP)

# evaluation metrics

In [ ]:
print('The acuuracy of the model = TP+TN / (TP+TN+FP+FN) = ',(TP+TN)/float(TP+TN+FP+FN),'\n\n',

'The Miss-classification = 1-Accuracy = ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n\n',

'Sensitivity or True Positive Rate = TP / (TP+FN) = ',TP/float(TP+FN),'\n\n',

'Specificity or True Negative Rate = TN / (TN+FP) = ',TN/float(TN+FP),'\n\n',

'Positive Predictive value = TP / (TP+FP) = ',TP/float(TP+FP),'\n\n',

'Negative predictive Value = TN / (TN+FN) = ',TN/float(TN+FN),'\n\n',

'Positive Likelihood Ratio = Sensitivity / (1-Specificity) = ',sensitivity/(1-specificity),'\n\n',

'Negative likelihood Ratio = (1-Sensitivity) / Specificity = ',(1-sensitivity)/specificity)

**From the above statistics it is clear that the model is highly sensitive than specific. The positive values are predicted more accurately than the negative i.e, it is predicting people who will not make purchase more accurately than who make purchase.**

### <font color= Blue>Predicted probabilities of  0 (will not make purchase) and 1 ( purchase)  for the test data with a default classification threshold of 0.5 <font>

In [ ]:
y_pred_prob=logreg.predict_proba(x_test)[:,:]
y_pred_prob_df=pd.DataFrame(data=y_pred_prob, columns=['Prob of no purchase (0)','Prob of purchase (1)'])
y_pred_prob_df.head()

- A common way to visualize the trade-offs of different thresholds is by using an ROC curve, a plot of the true positive rate ( true positives/ total  positives) versus the false positive rate ( false positives /total  negatives) for all possible choices of thresholds.
- A model with good classification accuracy should have significantly more true positives than false positives at all thresholds. 
- The optimum position for roc curve is towards the top left corner where the specificity and sensitivity are at optimum levels

In [ ]:
roc_auc_score(y_test,y_pred)

### Area Under The Curve (AUC)

- The area under the ROC curve quantifies model classification accuracy ; the higher the area, the greater the disparity between true and false positives, and the stronger the model in classifying members of the training dataset.
- An area of 0.5 corresponds to a model that performs no better than random classification and a good classifier stays as far away from that  as possible. An area of 1 is ideal. 
- The closer the AUC to 1 the better.

In [ ]:
# results matrix
df_results = pd.DataFrame(columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])

In [ ]:
# itereation results
description = "Base logit model"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,y_pred),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

In [ ]:
print(classification_report(y_test,y_pred))

# threshold
- Since the model is predicting people who dont purchase too many type II errors is not advisable. 
- A False Negative ( ignoring the probability of purchase when there actualy is one) is more dangerous than a False Positive in this case.
- Hence inorder to increase the sensitivity,  threshold can be lowered.

In [ ]:
from sklearn.preprocessing import binarize
for i in range(1,5):
  cm2 = 0
  y_pred_prob_yes = logreg.predict_proba(x_test)
  y_pred2 = np.where(y_pred_prob_yes[:, 1] > i/10, 1, 0)
  cm2 = confusion_matrix(y_test, y_pred2)
  print('With', i/10, 'threshold the Confusion Matrix is', '\n', cm2, '\n',
        'with', cm2[0, 0] + cm2[1, 1], 'correct predictions and', cm2[1, 0], 'Type II errors (False Negatives)', '\n\n',
        'Sensitivity:', cm2[1, 1] / (float(cm2[1, 1] + cm2[1, 0])), 'Specificity:', cm2[0, 0] / (float(cm2[0, 0] + cm2[0, 1])), '\n\n\n')
    

In [ ]:
# ROC curve

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_yes[:,1])
plt.plot(fpr,tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for online shoppers classifier')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('True positive rate (Sensitivity)')
plt.grid(True)

In [ ]:
## 1)Here,we can see that even decreasing threshold it is causing type 1 error.

## 2)we can use the default threshold 0.5 

## we will use kfold cv to decide which model performs better

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
DT=DecisionTreeClassifier()
RF=RandomForestClassifier(criterion='entropy',n_estimators=10)
Bagged=BaggingClassifier(n_estimators=100)
AB_RF=AdaBoostClassifier(base_estimator=RF,n_estimators=150)
GBoost=GradientBoostingClassifier(n_estimators=300)
KNN=KNeighborsClassifier(n_neighbors=9,weights='distance')

In [ ]:
models = [] 
models.append(('DT',DT))
models.append(('RandomForest',RF))
models.append(('Bagged',Bagged))
models.append(('AdaBoostRF',AB_RF))
models.append(('GradientBoost',GBoost))
models.append(('KNN',KNN))

In [ ]:
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=3,shuffle=True,random_state=0)
    cv_results = model_selection.cross_val_score(model,X,y,cv=kfold,scoring='f1_weighted')
    results.append(np.sqrt(np.abs(cv_results)))
    names.append(name)
    print("%s: %f (%f)" % (name, 1-np.mean(cv_results),np.std(cv_results,ddof=1)))


## We can see ensemble models are performing better like bagging and gradientboost and random forest.so lets try building model with them.

In [ ]:
rf=RandomForestClassifier().fit(x_train,y_train)
y_pred_rf=rf.predict(x_test)
print(classification_report(y_test,y_pred_rf))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_rf)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

TP=cm[0,0]
TN=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

In [ ]:
# itereation results
description = "Random Forest Classifier"  #change the name of models
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred_rf),2)
recall = round(recall_score(y_test,y_pred_rf),2)
accuracy = round(accuracy_score(y_test,y_pred_rf),2)
f1 = round(f1_score(y_test,y_pred_rf),2)
auc = round(roc_auc_score(y_test,y_pred_rf),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

In [ ]:
ada=AdaBoostClassifier().fit(x_train,y_train)
y_pred_ada=ada.predict(x_test)
print(classification_report(y_test,y_pred_ada))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_ada)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

TP=cm[0,0]
TN=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

In [ ]:
# itereation results
description = "Ada Boost Classifier"  #change the name of models
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred_ada),2)
recall = round(recall_score(y_test,y_pred_ada),2)
accuracy = round(accuracy_score(y_test,y_pred_ada),2)
f1 = round(f1_score(y_test,y_pred_ada),2)
auc = round(roc_auc_score(y_test,y_pred_ada),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

In [ ]:
gbc=GradientBoostingClassifier().fit(x_train,y_train)
y_pred_gbc=gbc.predict(x_test)
print(classification_report(y_test,y_pred_ada))

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred_gbc)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

TP=cm[0,0]
TN=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

In [ ]:
# itereation results
description = "Gradient Boost Classifier"  #change the name of models
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred_gbc),2)
recall = round(recall_score(y_test,y_pred_gbc),2)
accuracy = round(accuracy_score(y_test,y_pred_gbc),2)
f1 = round(f1_score(y_test,y_pred_gbc),2)
auc = round(roc_auc_score(y_test,y_pred_gbc),2)

df_results = pd.concat([df_results,
                        pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy','F1-score','ROC AUC'])
                                  ], axis=0)

df_results

## with gradient boosting we got very good results and we can also redce our missclassifications .further,we use imbalanced techniques so that bias can be reduced and model can predict more accurately and show disparity between each class .